In [3]:
import torch
import torch.nn as nn
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

### Building CNN class

In [2]:
class MyCNN_alexnet(nn.Module):
    def __init__(self):
        super(MyCNN_alexnet, self).__init__()
        self.conv1 =  nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0) # 55x55x96 (output)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2) # 27x27x256
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride = 1, padding = 1) # 27x27x384
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride = 1, padding = 1) # 27x27x384
        self.conv5 = nn.Conv2d(in_channels= 384, out_channels=256, kernel_size =3, stride = 1, padding=1 ) # 27x27x256
        self.fc1 = nn.Linear(in_features=256, out_features=4096) # 4096
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096) # 4096
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=4096, out_features=1000) # 1000
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x) # 55x55x96
        x = self.relu(x)
        x = self.pool(x) # 27x27x96
        x = self.conv2(x) # 27x27x256   
        x = self.relu(x)
        x = self.pool(x) # 13x13x256
        x = self.conv3(x) # 13x13x384
        x = self.relu(x)
        x = self.conv4(x) # 13x13x384
        x = self.relu(x)
        x = self.conv5(x) # 13x13x256   
        x = self.relu(x)
        x = self.pool(x) # 6x6x256
        x = x.view(-1, 256)
        x = self.fc1(x) # 4096
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x) # 4096
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x) # 1000
        x = torch.nn.functional.softmax(x, dim=1)
        return x

In [6]:
model = MyCNN_alexnet()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
           Linear-14                 [-